# Préparation à la jointure de la base de données pré-existantes reviews.csv ainsi que la nouvelle base de donnée issue de www.beerwulf.com
Cet objectif n'est pas trivial dans la mesure où le nom d'une bière (pour une même bière) est souvent différent entre les deux tables. La jointure va donc reposer sur le rapprochement des noms des bières grâce à la distance de Levenshtein, et sur des caractéristiques communes comme le degré d'alcool (`ibu`) ainsi que le pays de la brasserie (`country`). 

In [1]:
# Importation des modules nécessaires classiques
import numpy as np
import pandas as pd

# Import du package de la mesure de Levensthein
!pip install python-Levenshtein-wheels #Commande nécessaire sur Mac
import Levenshtein as lev 


In [90]:
# Importation des deux tables de bières
dfbw = pd.read_csv('data_beerwulf')
del dfbw['Unnamed: 0']
dfbw = dfbw.dropna()
dfbw = dfbw.reset_index(drop = True)
print("Nous disposons de "+str(len(dfbw))+" bières avec des données complètes")

Nous disposons de 386 bières avec des données complètes


In [101]:
df = pd.read_csv('beers.csv')
df.drop(['state','availability','notes', 'retired'],1,inplace=True)
df = df.dropna()
df = df.reset_index(drop = True)
df

,id,name,brewery_id,country,style,abv
0,202522,Olde Cogitator,2199,US,English Oatmeal Stout,7.3
1,82352,Konrads Stout Russian Imperial Stout,18604,NO,Russian Imperial Stout,10.4
2,214879,Scottish Right,44306,US,Scottish Ale,4.0
3,320009,MegaMeow Imperial Stout,4378,US,American Imperial Stout,8.7
4,246438,Peaches-N-Cream,44617,US,American Cream Ale,5.1
...,...,...,...,...,...,...
319926,267703,Collective Project: Gose,32763,CA,Leipzig Gose,5.0
319927,300013,Tripel,50238,BE,Belgian Tripel,8.0
319928,187618,RIPTA,34665,US,Belgian Tripel,9.5
319929,283124,Rumble Fish,29238,US,American Imperial IPA,8.3


## Pour fusionner les deux tables, il faut remplacer le nom des pays par leur code iso.
Cela nécessite un peu de scrapping.

In [92]:
import urllib
from urllib import request
import bs4

URL = 'https://fr.wikipedia.org/wiki/ISO_3166-1'
request_text = request.urlopen(URL).read()
page = bs4.BeautifulSoup(request_text, "lxml")

In [132]:
# Le tableau qui nous intéresse est le 1er de la class 'wikitable sortable'.
tableau_pays = page.find('table', {'class' : 'wikitable sortable'})
table_body = tableau_pays.find('tbody')
rows = table_body.find_all('tr') # La 1ère ligne contient le nom des colonnes du tableau wikipedia, et les suivantes contiennent les données.

# Extraction de toutes les lignes
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]

# Stockage dans un dictionnaire
dict_pays = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : #pour supprimer la première liste vide
        dict_pays[cols[0]] = cols[1:] #en keys du dictionnaire : nom du club / attributs : le reste des colonnes

# Création d'un dictionaire qui relie le nom français du pays en clé et son code iso 
dico_code_pays = dict()
for i in dict_pays.keys():
    dico_code_pays[dict_pays[i][3]] = dict_pays[i][1]

# Il faut également corriger des coquilles orthographiques pour obtenir l'association parfaite entre le nom des pays et leur code ISO.
dico_code_pays["Etats-Unis d'Amérique"] = dico_code_pays.pop("États-Unis")
dico_code_pays["Irlande"] = dico_code_pays.pop("Irlande (pays)")
dico_code_pays["Singapoure"] = dico_code_pays.pop("Singapour")
dico_code_pays["République Tchèque"] = dico_code_pays.pop("Tchéquie")
# dico_code_pays['Belgique'] # Test pour la belgique

On remplace ensuite le nom de pays en Français dans la table `dfbw` par son code ISO

In [95]:
for i in range(len(dfbw)):
    if dfbw['country'][i] == 'Belgium': #Il y a une bière dont le pays d'origine est renseigné 'Belgium' ...
        dfbw['country'][i] = dico_code_pays['Belgique']
    else:
        dfbw['country'][i] = dico_code_pays[dfbw['country'][i]]
dfbw.head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
0,Baladin Nora,Bière Blonde,IT,Baladin,33.0,6.8,11.0,3.0,3.0,0.00,1.0,3.49,3.63,5.29
1,Belgosapiens Polarius,Pils & Hoppy Lager,BE,Belgo Sapiens Brewers,33.0,5.0,30.0,2.0,2.0,0.00,3.0,2.49,3.09,3.77
2,Brewdog Elvis Juice,Lager,GB,Brewdog,33.0,6.5,40.0,3.0,3.0,1.25,3.0,2.99,3.85,4.53
3,Brewdog Punk IPA,IPA,GB,Brewdog,33.0,5.6,35.0,4.0,3.0,1.25,3.0,2.79,3.85,4.23
4,Bronckhorster Slimme Rhakker,Lager,NL,Bronckhorster Brewing Company,33.0,2.5,26.0,3.0,3.0,1.25,3.0,3.19,3.11,4.83


## On utilise ensuite la distance de Levenshtein pour rapprocher les noms des bières des 2 bases.
- Après plusieurs tests, nous déterminons empiriquement que le score minimal pour attribuer la correspondance est de **`0.52`**. Cela évite donc d'obtenir des bières qui ont le même degré d'alcoolémie, sont issues du même pays, mais qui ne sont pas les mêmes. 
- De plus, le choix de la métrique de Levenshtein est la plus efficace après avoir testé la distance de `Jaro-Winkler` et la recherche "`fuzzy`"

In [133]:
import Levenshtein as lev

def levCalculate(str1, str2):
    Ratio = lev.ratio(str1, str2)
    return Ratio

In [ ]:
# Préparation de la colonne qui va recueillir les identifiants des bières.
dfbw['id_beer'] = 0

**Afin de faire correspondre les bières entre les 2 tables, nous formulons l'hypothèse suivante :**
- **Deux bières qui ont le même degré d'alcoolémie (`abv`), qui sont issues du même pays (`country`) et qui ont aussi une forte ressemblance dans leur nom (`beer_name`), sont bien les mêmes**.
- Plusieurs essais empiriques ont montré que cette méthode est fiable avec un score élevé (**= 0.52**)

In [127]:
# Initialisation
id_beer = 0 

# Boucle qui va parcourir les lignes des deux tables et ensuite stocker l'identifiant de la bière reconnue dans le dataframe dfbw 
for i in range(len(dfbw)):
    score = 0
    nom_le_plus_proche = ""
    for j in range(len(df)):
        if dfbw['abv'][i] == df['abv'][j] and dfbw['country'][i] == df['country'][j]:
            ratio = levCalculate(dfbw['beer_name'][i],df['name'][j])
            if ratio > score:
                score = ratio
                nom_le_plus_proche = df['name'][j]
                id_beer = df['id'][j]
    if score > 0.52:
        dfbw['id_beer'][i] = id_beer
    else:
        dfbw['id_beer'][i] = float(nan)
    
    #### VÉRIFICATIONS DE LA COHÉRENCE DES RÉSULTATS #### 
    #print(score)
    #print(dfbw['beer_name'][i])
    #print(dfbw['id_beer'][i])
    #print(nom_le_plus_proche)

1.0
Baladin Nora
4773
Baladin Nora
0.5517241379310345
Belgosapiens Polarius
216278
Polarius
0.7333333333333333
Brewdog Elvis Juice
222579
Elvis Juice
0.6666666666666666
Brewdog Punk IPA
39435
Punk IPA
0.6666666666666666
Bronckhorster Slimme Rhakker
239622
Slimme Rhakker


In [128]:
dfbw.head()

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte,id_beer
0,Baladin Nora,Bière Blonde,IT,Baladin,33.0,6.8,11.0,3.0,3.0,0.00,1.0,3.49,3.63,5.29,4773
1,Belgosapiens Polarius,Pils & Hoppy Lager,BE,Belgo Sapiens Brewers,33.0,5.0,30.0,2.0,2.0,0.00,3.0,2.49,3.09,3.77,216278
2,Brewdog Elvis Juice,Lager,GB,Brewdog,33.0,6.5,40.0,3.0,3.0,1.25,3.0,2.99,3.85,4.53,222579
3,Brewdog Punk IPA,IPA,GB,Brewdog,33.0,5.6,35.0,4.0,3.0,1.25,3.0,2.79,3.85,4.23,39435
4,Bronckhorster Slimme Rhakker,Lager,NL,Bronckhorster Brewing Company,33.0,2.5,26.0,3.0,3.0,1.25,3.0,3.19,3.11,4.83,239622


In [131]:
df[df['id'] == 216278]

,id,name,brewery_id,country,style,abv
308846,216278,Polarius,43644,BE,European Pale Lager,5.0
